# Imports

In [178]:
import pandas as pd
import heapq
from collections import defaultdict
from operator import itemgetter
from lightfm.data import Dataset
from lightfm import lightfm
from lightfm.evaluation import auc_score
import numpy as np

In [2]:
olist_reviews = pd.read_csv("../data/processed/olist_order_reviews_reduced.csv")
olist_products = pd.read_csv("../data/processed/olist_products_dataset_english.csv")
olist_orders = pd.read_csv("../data/processed/olist_orders_reduced.csv")
olist_customers = pd.read_csv("../data/processed/olist_customers_dataset.csv")
olist_orders_items = pd.read_csv("../data/processed/olist_order_items_reduced.csv")

In [3]:
olist_products.head()

,Unnamed: 0,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,housewares


In [4]:
olist_reviews.head()

,Unnamed: 0,review_id,order_id,review_score
0,0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4
1,1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5
2,2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5
3,3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5
4,4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5


In [5]:
olist_orders.head()

,Unnamed: 0.1,Unnamed: 0,order_id,customer_id,order_status,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purchase_month,purchase_time
0,0,0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,10,night
1,1,1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,7,night
2,2,2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,8,morning
3,3,3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,11,evening
4,4,4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,2,afternoon


In [6]:
olist_customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


# Data preparation

In [162]:
def prepare_product_features_df(olist_reviews_df, olist_products_df, olist_order_items_df):
    merged_review_order =  olist_reviews.merge(right=olist_orders_items, how="inner", on="order_id" )
    merged_review_order_products = merged_review_order.merge(right=olist_products, how="inner", on="product_id")
    product_df = merged_review_order_products[["review_score", "price", "product_category_name_english", "product_name_lenght", "product_description_lenght", "product_weight_g", "product_id"]]
    return product_df
product_df = prepare_product_features_df(olist_reviews, olist_products, olist_orders_items)

In [171]:
def prepare_customer_features_df(olist_customers, olist_orders, olist_reviews):
    olist_customers_orders = olist_customers.merge(right=olist_orders, how='inner', on='customer_id')
    customers_orders = olist_customers_orders.merge(right=olist_orders_items, how='inner', on='order_id')
    customers = customers_orders.merge(right=olist_reviews, how='inner', on="order_id")
    return customers
customers = prepare_customer_features_df(olist_customers, olist_orders, olist_reviews)

In [18]:
def generate_feature_list(dataframe, features_name):
    """
    Generate features list for mapping 

    Parameters
    ----------
    dataframe: Dataframe
        Pandas Dataframe for Users or Q&A. 
    features_name : List
        List of feature columns name avaiable in dataframe. 
        
    Returns
    -------
    List of all features for mapping 
    """
    features = dataframe[features_name].apply(
        lambda x: ','.join(x.map(str)), axis=1)
    features = features.str.split(',')
    features = features.apply(pd.Series).stack().reset_index(drop=True)
    return features

In [154]:
customer_features_list = generate_feature_list(customers, ['customer_city'])
product_features_list = generate_feature_list(product_df, ['price', "product_category_name_english", "product_name_lenght", "product_description_lenght","product_weight_g"])

In [21]:
dataset = Dataset()
dataset.fit(items=set(product_df['product_id']), users=set(customers['customer_unique_id']), item_features=product_features_list, user_features=customer_features_list)

In [179]:
customers['weight'] = customers['review_score'] - 3
customer_product_interaction = list(zip(customers.customer_unique_id, customers.product_id, customers.weight))

In [180]:
interactions, weights = dataset.build_interactions(data=customer_product_interaction)

In [181]:
def func(x):
    return ','.join(x.map(str))

In [182]:
def create_features (dataframe, features_name, id_col):
    features = dataframe[features_name].apply(func, axis=1)
    features = features.str.split(',')
    features = list(zip(dataframe[id_col], features))
    return features

In [ ]:
customer_fea = create_features(customers, ["customer_city"], "customer_unique_id")
customer_features = dataset.build_user_features(data=customer_fea)

In [32]:
product_fea = create_features(product_df, ['price', "product_category_name_english", "product_name_lenght", "product_description_lenght","product_weight_g"], 'product_id')
product_features = dataset.build_item_features(data=product_fea)

# Model training

In [34]:
model = lightfm.LightFM(no_components=150, learning_rate=0.05, loss='warp', random_state=2022)
model.fit(interactions=interactions, item_features=product_features, user_features=customer_features, sample_weight=weights, epochs=5, verbose=True, num_threads=16)

In [133]:
from lightfm.evaluation import auc_score
import numpy as np

In [38]:
def calculate_auc_score(lightfm_model, interactions_matrix, 
                        question_features, professional_features): 
    """
    Measure the ROC AUC metric for a model. 
    A perfect score is 1.0.

    Parameters
    ----------
    lightfm_model: LightFM model 
        A fitted lightfm model 
    interactions_matrix : 
        A lightfm interactions matrix 
    question_features, professional_features: 
        Lightfm features 
        
    Returns
    -------
    String containing AUC score 
    """
    score = auc_score( 
        lightfm_model, interactions_matrix, 
        item_features=question_features, 
        user_features=professional_features, 
        num_threads=16).mean()
    return score

In [39]:
calculate_auc_score(model, interactions, product_features, customer_features)

0.7436696

# Top N recommendation

In [137]:
dataset.mapping()[1]['c64ede6d0ae8901b1b6fb03528c1b7e6']

82760

In [88]:
product_df['product_id'].nunique()

32789

In [112]:
len(dataset.mapping()[2].values())

32789

In [132]:
score =model.predict(10000, np.arange(32789))

In [134]:
df_scores = pd.DataFrame()
df_scores['scores']= score
df_scores['productID'] = dataset.mapping()[2].keys()

In [138]:
customers[customers['customer_unique_id'] == "c64ede6d0ae8901b1b6fb03528c1b7e6"]

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,Unnamed: 0.1,Unnamed: 0_x,order_id,order_status,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,purchase_month,purchase_time,Unnamed: 0_y,order_item_id,product_id,seller_id,price,freight_value
112618,184e0c2cfc746789643521df0e9ff904,c64ede6d0ae8901b1b6fb03528c1b7e6,68660,sao miguel do guama,PA,62943,62943,45b3000bcd10464ac178f32cd783fc83,delivered,2017-12-11 20:29:02,2018-01-05 17:59:38,2018-01-22 00:00:00,12,morning,30720,1,bbf975bffd2ae9ee52f513ae5c8a4b27,04aba03279157f6d4e0fe8ccaf21963c,250.0,54.11


In [135]:
df_scores.sort_values("scores", ascending=False)

,scores,productID
4079,7.636342,a298a105818dce6878b787e4af6cff7d
13395,4.329013,698b3ddae2f0b80c2a48fb40624ca4e4
31786,3.848917,44a5d24dd383324a421569ca697b13c2
11456,3.513630,764292b2b0f73f77a0272be03fdd45f3
11640,3.355220,f3ce47ccf98da2906f6b8b1387618e07
...,...,...
17207,-4.890227,b36f3c918c91478c4559160022d3f14e
23114,-5.327689,89b121bee266dcd25688a1ba72eefb61
4973,-6.734810,c230b471b7e21ff9060e68ee154afd70
22168,-8.605264,ee3d532c8a438679776d222e997606b3


In [146]:
product_df[product_df["product_id"] == "c230b471b7e21ff9060e68ee154afd70"]

,review_score,price,product_category_name_english,product_name_lenght,product_description_lenght,product_weight_g,product_id
27955,1,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70
27956,1,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70
27957,1,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70
27958,1,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70
27959,1,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70
27960,1,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70
27961,2,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70
27962,1,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70
27963,1,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70
27964,1,69.99,NaN,NaN,NaN,200.0,c230b471b7e21ff9060e68ee154afd70


In [110]:
len(dataset.mapping()[2].keys())

32789

In [97]:
pd.DataFrame(dataset.mapping()[3].keys())

,0
0,2be252f11ad496bf9d981165c040af65
1,b5285354b7300aa056db9c3b73e50cd0
2,0df2a1aed6dc8c10577115fa9e4b6de0
3,efd037bbacbb4bbf39f4ecb8ba8714cf
4,feb593f17c2f51af2932dbba3e99f1cb
...,...
41995,257.87
41996,174.51
41997,132.45
41998,37.38


In [ ]:
import numpy as np
def sample_recommendation(model, data, user_ids):
    

    n_users, n_items = data['train'].shape

    for user_id in user_ids:
        known_positives = data['item_labels'][data['train'].tocsr()[user_id].indices]
        
        scores = model.predict(user_id, np.arange(n_items))
        top_items = data['item_labels'][np.argsort(-scores)]
        
        print("User %s" % user_id)
        print("     Known positives:")
        
        for x in known_positives[:3]:
            print("        %s" % x)

        print("     Recommended:")
        
        for x in top_items[:3]:
            print("        %s" % x)
        
sample_recommendation(model, data, [3, 25, 450]) 